Los modelos ya entrenados como GPT (la P es de pre-entrenado) se emplean mayormente como motores de inferencia o generación de muestras (texto). Ya vimos que los modelos de aprendizaje automático presentan estas dos etapas: 

* **entrenamiento**: donde se ajustan los pesos dados unos datos ejemplo del proceso que se quiere emular.
* **inferencia**: donde se usa el modelo entrenado presentandole una información de entrada para que genere su resultado.

De forma habitual usamos los modelos del lenguaje o LLMs a través de proveedores que realizan la inferencia en sus máquinas y nos devuelven el resultado. Esto hace que de cara a ajustar este resultado muchas empresas hayan invertido tiempo en el ajuste de la entrada (prompting, RAG, contexto-engineering,...). Sin embargo veremos que podemos ir un poco más allá.

## Modelos abiertos

Los modelos de código abierto o pesos abiertos nos permiten descargar tanto su estructura como sus pesos para que así nosotros podamos albergar ese servicio de inferencia.

Existe diversas opciones:

* Llama de [Meta](https://ai.meta.com/blog/meta-llama-3-1/)
* Qwen de [alibaba](https://qwen.ai/home)
* gpt-oss de [OpenAI](https://openai.com/es-ES/index/introducing-gpt-oss/)

Aunque casi todas las encontraremos en [HuggingFace.co](https://huggingface.co/models?pipeline_tag=text-generation&sort=trending). Estos modelos presentan diferencias en su arquitectura así como pesos ya que habrán sido entrenados con distintos juegos de datos.

![](https://sebastianraschka.com/images/blog/2025/from-gpt-2-to-gpt-oss/3.png)

https://sebastianraschka.com/blog/2025/from-gpt-2-to-gpt-oss.html

Ejecutar estos monstruos sin embargo no es cosa sencilla. Requieren una arquitectura demandante con casi obligatoriedad de emplear varias GPUs para aligerar el tiempo de producción de resultados.Podemos aligerar parte de la carga cambiando el tipo de dato que representa los pesos. Si pasáramos de emplear 16 a 8 bites, por ejemplo, reduciríamos a la mitad el peso específico del modelo, lo cual aligera casi cualquier aspecto posterior aunque también impacta en la perdida de precisión. Es lo que comúnmente se conoce como **cuantización**.

El proceso de inferencia tiene dos fases muy diferenciadas:

* **Prefill**: Donde se carga la información de entrada condicionando la sección generativa
* **Decoding**: Donde se empiezan a generar los tokens de forma secuenciada.

Existen maneras de optimizar estos dos procesos buscando la mejora de tiempos clave como el tiempo hasta el primer token (TTFT), tiempo de respuesta por usuario (TPOT) o la capacidad de salida del modelo (Throughput).

![](https://www.aleksagordic.com/blog/vllm/latency_diagram.png)

Se procura enfocar en ambas fases para establecer las mejores arquitecturas que permiten disminuir el tiempo de carga de memoria de GPU a registro y así jugar con la información que le es servida a la GPU en lotes. Particularmente en la fase de prefill donde se calculan las matrices de atención para poder iniciar el proceso de generación existen variantes que nos permiten cachear parcialmente resultados (KV caching) o paginar el mecanismo de atención que aceleran bastante el proceso.

Formula para KV caching: 

```
Total KV cache size (bytes) = batch_size × sequence_length × 2 × num_layers × hidden_size × precision_in_bytes
```

Atención paginada

![](https://www.adaline.ai/_next/image?url=https%3A%2F%2Fa-us.storyblok.com%2Ff%2F1023026%2F1192x628%2Fec03d2cc7e%2Fpagedattention.png&w=1920&q=75)

Referencia: https://www.databricks.com/blog/llm-inference-performance-engineering-best-practices

Existen varias opciones para poder ejecutar estos modelos en máquinas locales que incluyen parte de estas mejoras

* [Ollama](https://ollama.com/): Sencillo de usar.
* [SGLang](https://github.com/sgl-project/sglang): Planteado principalmente con mejoras para sistemas multi-modo (LLM, VLM) 
* [vLLM](https://docs.vllm.ai/en/latest/): Motor de inferencia que prima el throughput e implementa varias de las mejoras arriba indicadas
* [TensorRT-LLM](https://docs.nvidia.com/tensorrt-llm/index.html): Específico de NVIDIA para la utilización de sus Tensor Cores.

Deberemos elegir uno que presenta la versatilidad y facilidad de uso que necesitamos.

> NOTA
> Las dependencias de estas librerías son extras que deberemos activar mediante `uv sync --group serving`

In [1]:
from vllm import LLM, SamplingParams

prompts = [
    "Hello, my name is",
    "The president of the United States is",
]

# Parámetros de muestreo
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

# Modelo
llm = LLM(model="facebook/opt-125m") #"TinyLlama/TinyLlama-1.1B-Chat-v1.0"

outputs = llm.generate(prompts, sampling_params)

INFO 09-06 12:25:21 [__init__.py:241] Automatically detected platform cuda.
INFO 09-06 12:25:22 [utils.py:326] non-default args: {'model': 'facebook/opt-125m', 'disable_log_stats': True}
INFO 09-06 12:25:28 [__init__.py:711] Resolved architecture: OPTForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 09-06 12:25:28 [__init__.py:1750] Using max model len 2048
INFO 09-06 12:25:29 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_0 pid=353318) INFO 09-06 12:25:30 [core.py:636] Waiting for init message from front-end.
(EngineCore_0 pid=353318) INFO 09-06 12:25:30 [core.py:74] Initializing a V1 LLM engine (v0.10.1.1) with config: model='facebook/opt-125m', speculative_config=None, tokenizer='facebook/opt-125m', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend='')

Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_0 pid=353318) INFO 09-06 12:25:32 [default_loader.py:262] Loading weights took 0.25 seconds
(EngineCore_0 pid=353318) INFO 09-06 12:25:33 [gpu_model_runner.py:2007] Model loading took 0.2389 GiB and 1.064055 seconds
(EngineCore_0 pid=353318) INFO 09-06 12:25:34 [backends.py:548] Using cache directory: /home/iraitz/.cache/vllm/torch_compile_cache/6d1eecaf73/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_0 pid=353318) INFO 09-06 12:25:34 [backends.py:559] Dynamo bytecode transform time: 1.36 s
(EngineCore_0 pid=353318) INFO 09-06 12:25:35 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 0.617 s
(EngineCore_0 pid=353318) INFO 09-06 12:25:35 [monitor.py:34] torch.compile takes 1.36 s in total
(EngineCore_0 pid=353318) INFO 09-06 12:25:36 [gpu_worker.py:276] Available KV cache memory: 6.24 GiB
(EngineCore_0 pid=353318) INFO 09-06 12:25:36 [kv_cache_utils.py:849] GPU KV cache size: 181,728 tokens
(EngineCore_0 pid=353318) INFO 0

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:00<00:00, 121.13it/s]


(EngineCore_0 pid=353318) INFO 09-06 12:25:37 [gpu_model_runner.py:2708] Graph capturing finished in 1 secs, took 0.21 GiB
(EngineCore_0 pid=353318) INFO 09-06 12:25:37 [core.py:214] init engine (profile, create kv cache, warmup model) took 4.08 seconds
INFO 09-06 12:25:38 [llm.py:298] Supported_tasks: ['generate']


Adding requests:   0%|          | 0/2 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [3]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Prompt: 'Hello, my name is', Generated text: ' Joel, my dad is my friend and we are in a relationship. I am'
Prompt: 'The president of the United States is', Generated text: ' engaged in a long-running war with Russian President Vladimir Putin.\n\nThe'


In [4]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.1, top_k=2)

outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Adding requests:   0%|          | 0/4 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Prompt: 'Hello, my name is', Generated text: ' J.C. and I am a student at the University of California, Berkeley'
Prompt: 'The president of the United States is', Generated text: " a racist.\nHe's a racist because he's a racist.\nHe"
Prompt: 'The capital of France is', Generated text: ' the capital of the French Republic.\n\nThe capital of France is the capital'
Prompt: 'The future of AI is', Generated text: ' in the hands of the people.\n\nThe future of AI is in the'


Podemos ver como afecta un prefijo extenso antes de cada prompt (por defecto 16 tokens) en la carga y reutilización de los bloques de memoria.

In [5]:
%%time

# no_prefix

prompts = [
    "Hello, my name is",
    "The president of the United States is",
]

sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

outputs = llm.generate(prompts[0], sampling_params)
outputs = llm.generate(prompts[1], sampling_params)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

CPU times: user 27.5 ms, sys: 4.13 ms, total: 31.6 ms
Wall time: 105 ms


In [7]:
%%time

long_prefix = "<a piece of text that is encoded into more than block_size tokens which could vary depending on the model being used and the particular configuration of it>"

prompts = [
    "Hello, my name is",
    "The president of the United States is",
]

sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

outputs = llm.generate(long_prefix + prompts[0], sampling_params)
outputs = llm.generate(prompts[1], sampling_params)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

CPU times: user 33.8 ms, sys: 1.26 ms, total: 35 ms
Wall time: 107 ms


Estos controles de bajo nivel de la inferencia nos permite actuar a nivel de vocabulario y construir un resultado dentro del vocabulario esperado.

In [8]:
from vllm.sampling_params import GuidedDecodingParams

prompts = [
    "This sucks",
    "The weather is beautiful",
]

guided_decoding_params = GuidedDecodingParams(choice=["Positive", "Negative"]) # Palabras admitidas a generar
sampling_params = SamplingParams(guided_decoding=guided_decoding_params)
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Adding requests:   0%|          | 0/2 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Prompt: 'This sucks', Generated text: 'Positive'
Prompt: 'The weather is beautiful', Generated text: 'Positive'


Cuando exista una demanda alta de peticiones, deberemos evaluar la opción de realizar inferencia en sistemas multi-GPU. Esto es casi obligatorio para el entrenamiento (al menos para el entrenamiento desde cero) pero en caso de mucha demanda, podemos beneficiarnos de esta arquitectura modular que presentan las LLMs y las opciones de cacheo de los mecanismos de atención.

![](https://www.aleksagordic.com/blog/vllm/server_setup.png)

Aquí tenéis una referencia en detalle sobre vLLM: https://www.aleksagordic.com/blog/vllm

Servir nuestro propios modelos internamente hace que podamos variar sus parametrizaciones o jugar con distintos modelos para propósitos concretos:

* Modelos rápidos y precisos en interacción con usuario
* Modelos más lentos y específicos para agentes autónomos

La gente de BentoLM se ha enfocado en que esta tarea sea algo más fácil de gestionar y nos ofrecen un servidor estandarizado para todas estas tareas, [OpenLLM](https://github.com/bentoml/OpenLLM).

Aún así podemos necesitar ajustar aún más nuestros modelos.

## Fine tuning

El fine tuning precisa de las capacidades anteriormente indicadas ya que vamos a ajustar o variar los pesos del modelo para orientarlos a un propósito concreto.

![](https://camo.githubusercontent.com/a17472f25db0af2e7a72700cf3e994b48a61405931b54111ed4d62cbe0371216/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f6d656e74616c2d6d6f64656c2e6a7067)

Referencia: https://github.com/rasbt/LLMs-from-scratch

Existen etapas diferenciadas pero un aspecto clave es poder recabar ejemplos del proceso en el que queremos que nuestro modelo _custom_ se especialice. Seguiremos los pasos del libro _LLMs-from-scratch_ en su capítulo 7.

![](https://camo.githubusercontent.com/6736ab7968f8da6bd6fc747de22ef9afa9d840373749005ce3e96fc6ead7ed8c/68747470733a2f2f73656261737469616e72617363686b612e636f6d2f696d616765732f4c4c4d732d66726f6d2d736372617463682d696d616765732f636830375f636f6d707265737365642f636861707465722d6f766572766965772d312e776562703f31)

Veamos si podemos seguir los pasos de este flujo usando Colab.

[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rasbt/LLMs-from-scratch/blob/main/ch07/01_main-chapter-code/ch07.ipynb)

Con estas capacidades podríamos entrenar todos los pesos de la red pero quizás solo necesitemos ajustarlos en nuestra dirección. Aquí es donde aparecen las técnicas más económicas ya que requieren un 0.1% a 1% del tiempo de un entrenamiento completo.

* **LoRA (Low Rank Adaptation)** reduce el número de parámetros a entrenar aproximando las matrices de pesos mediante productos de matrices de menor rango, lo que permite ajustes más eficientes y económicos del modelo.
* **Quantized LoRA**: Combina la cuantización de 4-bit con LoRA, reduciendo aún más la huella de memoria durante el entrenamiento mientras mantiene resultados similares a LoRA estándar. Permite fine-tuning de modelos más grandes en hardware más modesto.
* **Reinforcemenent Learning from Human Feedback (RLHF)** se trata de aprender y refinar basado en las puntuaciones tras la validación humana
* **Direct Preference Optimization (DPO)** ofrece un mecanismos similar al anterior pero mediante ajustes automatizados basados en entropía cruzada y descenso del gradient. Así podemos acortar los ciclos de feedback de los humanos.
* **Group Relative Policy Optimization (GRPO)**: Es una técnica destinada a modelos de razonamiento que son ajustados mediante técnicas de aprendizaje por refuerzo (RFT) similares a las anteriores.

Todas estas técnicas están ya implementadas en múltiples frameworks que nos abstraen del trabajo más técnico aunque deberemos como mínimo presentar el **conjunto de datos** y ofrecer una forma de **puntuar las respuestas del modelo**.

* [Axolotl](https://axolotl.ai/)
* [Unsloth (ejemplos)](https://docs.unsloth.ai/get-started/unsloth-notebooks)
* [Torchtune](https://github.com/pytorch/torchtune)
* [LlamaFactory](https://llamafactory.readthedocs.io/en/latest/)
* [Nemo-RL](https://docs.nvidia.com/nemo/rl/latest/index.html)

Más referencias en:

* https://github.com/rasbt/LLMs-from-scratch/tree/main
* https://bentoml.com/llm/getting-started/llm-fine-tuning
* https://github.com/Azure/azure-llm-fine-tuning/tree/main
* https://aiengineering.academy/ 
